## Sentiment Analysis

In [7]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from tqdm import tqdm

In [8]:
# Load dữ liệu đã tiền xử lý
df = pd.read_csv("data/2025-10-31/playlist_data_clean.csv")

In [9]:
from transformers import pipeline

sentiment_model = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

df["sentiment"] = df["combined_text"].apply(
    lambda x: sentiment_model(x[:512])[0]['label'] if isinstance(x, str) and len(x) > 0 else "neutral"
)

print(df["sentiment"].value_counts())


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

sentiment
neutral     48
positive     1
Name: count, dtype: int64


In [10]:
df.to_csv("data/2025-10-31/playlist_data_sentiment.csv", index=False, encoding="utf-8")
print("✅ Đã lưu kết quả phân tích cảm xúc!")

✅ Đã lưu kết quả phân tích cảm xúc!


In [11]:
df_clone = df.copy()

In [12]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return "neutral"
    score = sia.polarity_scores(text)["compound"]
    if score > 0.05:
        return "positive"
    elif score < -0.05:
        return "negative"
    else:
        return "neutral"

df_clone["sentiment"] = df_clone["combined_text"].apply(get_sentiment)
print(df_clone["sentiment"].value_counts())


LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - 'C:\\Users\\quocla/nltk_data'
    - 'c:\\Users\\quocla\\AppData\\Local\\Programs\\Python\\Python310\\nltk_data'
    - 'c:\\Users\\quocla\\AppData\\Local\\Programs\\Python\\Python310\\share\\nltk_data'
    - 'c:\\Users\\quocla\\AppData\\Local\\Programs\\Python\\Python310\\lib\\nltk_data'
    - 'C:\\Users\\quocla\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


## Classification

In [13]:

import pandas as pd

df = pd.read_csv("data/2025-10-31/playlist_data_clean.csv")


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

vectorizer = TfidfVectorizer(
    max_features=5000,
    stop_words='english'
)
X = vectorizer.fit_transform(df["combined_text"])

# Giả sử chia thành 2 cụm: 0 = nội dung tự nhiên, 1 = quảng cáo
kmeans = KMeans(n_clusters=2, random_state=42)
df["cluster"] = kmeans.fit_predict(X)


In [15]:
terms = vectorizer.get_feature_names_out()
order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]

for i in range(2):
    print(f"\n🔹 Cụm {i}:")
    top_terms = [terms[ind] for ind in order_centroids[i, :15]]
    print("Top keywords:", top_terms)



🔹 Cụm 0:
Top keywords: ['cân', 'giảm', 'nioslim', 'quảng', 'phim', 'cáo', 'tỳ', 'vị', 'mộc', 'thúc', 'mỡ', 'mc', 'hà', 'kẹo', 'kera']

🔹 Cụm 1:
Top keywords: ['bạn', 'có', 'thuốc', 'và', 'tăng', 'không', 'uống', 'cho', 'mua', 'giá', 'linh', 'được', 'hộp', 'bao', 'mình']


In [16]:
def label_cluster(row):
    if row["cluster"] == 0:
        return "ad_real"
    else:
        return "ad_fake"
df["ad_type"] = df.apply(label_cluster, axis=1)


In [19]:
print(df["ad_type"].value_counts())

ad_type
ad_fake    40
ad_real     9
Name: count, dtype: int64


In [17]:

df.to_csv("data/2025-10-31/playlist_data_clustered.csv", index=False, encoding="utf-8")
print("✅ Đã lưu kết quả phân cụm nội dung!")

✅ Đã lưu kết quả phân cụm nội dung!


## Merge 2 model

In [18]:
output_path2 = "data/2025-10-31/playlist_data_analyzed.csv"
df.to_csv(output_path2, index=False, encoding="utf-8")
print(f"💾 Đã lưu kết quả phân tích: {output_path2}")

💾 Đã lưu kết quả phân tích: data/2025-10-31/playlist_data_analyzed.csv


In [1]:
# ====================================================
# 🔧 CÀI ĐẶT & IMPORT
# ====================================================
!pip install -q pandas numpy tqdm scikit-learn underthesea torch transformers sentence-transformers

import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from transformers import pipeline

'pip' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# ====================================================
# 📂 ĐỌC DỮ LIỆU
# ====================================================
input_path = "data/2025-10-31/playlist_data_clean.csv"
df = pd.read_csv(input_path)
print("✅ Đã đọc file:", df.shape)

# Đảm bảo cột combined_text tồn tại
assert "combined_text" in df.columns, "⚠️ Thiếu cột 'combined_text' trong CSV!"

✅ Đã đọc file: (49, 17)


In [9]:
# ====================================================
# 🧠 1️⃣ PHÂN TÍCH CẢM XÚC (Sentiment Analysis - tiếng Việt)
# ====================================================

# Mô hình tiếng Việt tốt nhất hiện tại: "uitnlp/vietnamese-sentiment" hoặc "manhthang/vietnamese-sentiment-analysis"
sentiment_pipeline = pipeline(
    "text-classification",
    model="manhthang/vietnamese-sentiment-analysis",
    tokenizer="manhthang/vietnamese-sentiment-analysis"
)

def analyze_sentiment(text):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return "neutral"
    try:
        res = sentiment_pipeline(text[:512])[0]["label"].lower()
        if "pos" in res:
            return "positive"
        elif "neg" in res:
            return "negative"
        else:
            return "neutral"
    except Exception:
        return "neutral"

tqdm.pandas()
df["sentiment"] = df["combined_text"].progress_apply(analyze_sentiment)

print("📊 Phân bố cảm xúc:")
print(df["sentiment"].value_counts())

OSError: manhthang/vietnamese-sentiment-analysis is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [4]:
# ====================================================
# 🕵️‍♂️ 2️⃣ PHÂN CỤM PHÁT HIỆN QUẢNG CÁO GIẢ / THẬT (Unsupervised)
# ====================================================

# 🔸 Dùng mô hình embedding tiếng Việt mạnh: PhoBERT hoặc SentenceTransformer đa ngôn ngữ
print("🔍 Đang tạo embedding bằng SentenceTransformer (mô hình đa ngôn ngữ)...")
embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# Tạo vector cho mỗi video
embeddings = embed_model.encode(df["combined_text"].tolist(), show_progress_bar=True)

# 🔸 Clustering (KMeans)
NUM_CLUSTERS = 2  # 2 cụm: quảng cáo thật / giả
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42, n_init=10)
df["cluster"] = kmeans.fit_predict(embeddings)

# ====================================================
# 🧩 PHÂN TÍCH TỪ KHÓA MỖI CỤM ĐỂ ĐẶT TÊN
# ====================================================
vectorizer = TfidfVectorizer(max_features=3000, stop_words=None)
X_tfidf = vectorizer.fit_transform(df["combined_text"])
terms = vectorizer.get_feature_names_out()
order_centroids = kmeans.cluster_centers_.argsort(axis=1)[:, ::-1]

print("\n🔹 Top từ khóa mỗi cụm:")
for i in range(NUM_CLUSTERS):
    top_terms = [terms[ind] for ind in order_centroids[i, :15]]
    print(f"Cụm {i}:", top_terms)

🔍 Đang tạo embedding bằng SentenceTransformer (mô hình đa ngôn ngữ)...


No sentence-transformers model found with name sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2. Creating a new one with mean pooling.


SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/resolve/main/adapter_config.json (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)')))"), '(Request ID: bc4cbcf3-a80d-411a-8407-03772a9643de)')

In [ ]:
# ====================================================
# 🏷️ GÁN NHÃN DỰA TRÊN TỪ KHÓA (TỰ SUY DIỄN)
# ====================================================

# Nếu có các từ liên quan đến bán hàng / sản phẩm => có thể là quảng cáo thật
ad_keywords = ["mua", "shop", "link", "giảm", "sale", "đặt hàng", "sản phẩm", "official", "giá"]

def infer_ad_label(text, cluster_id):
    text_lower = str(text).lower()
    if any(k in text_lower for k in ad_keywords):
        return "ad_real"
    elif "giả" in text_lower or "fake" in text_lower or "lừa" in text_lower:
        return "ad_fake"
    else:
        # fallback bằng cluster nếu không có keyword
        return "ad_real" if cluster_id == 0 else "ad_fake"

df["ad_type"] = df.apply(lambda r: infer_ad_label(r["combined_text"], r["cluster"]), axis=1)

print("\n📊 Phân bố nhóm quảng cáo:")
print(df["ad_type"].value_counts())



In [ ]:
# ====================================================
# 💾 LƯU KẾT QUẢ
# ====================================================
output_path = "data/2025-10-31/playlist_data_analyzed_vi.csv"
df.to_csv(output_path, index=False, encoding="utf-8")
print(f"\n💾 Đã lưu kết quả phân tích tiếng Việt tại: {output_path}")

# ====================================================
# ✅ XEM TRƯỚC KẾT QUẢ
# ====================================================
print(df[["video_id", "sentiment", "ad_type", "combined_text"]].head(5))